Create a file with data interpolated from nearest grid points
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

Period-specific data
----

In [2]:
branch = "1pt5degC"
yr1 = "2071"
yr2 = "2080"
code = "BRCP26C5CNBDRD"

mon = 7
nEns = 11

suff = str(yr1)+"0101-"+str(yr2)+"1231"

File templates etc
---

In [10]:
# modelDataFN = "daily/b.e11.B20TRC5CNBDRD.f09_g16.001.cam.h1.TREFHT.19960101-20051231_sub.nc"
modelDataTemplate = branch+"/daily/b.e11.%s.f09_g16.1pt5degC.%03d.cam.h1.TREFHT.%s0101-%s1231_sub.nc"
modelMetaFN = "cesmlw_geog_sub.nc"
newNCFN = "cesmlw_tas_interp_"+suff+".nc"

Load/define CESM LW metadata
----

In [4]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

1d dataset


Load AWS metadata
----

In [5]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [6]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )

Process TREFHT data
====

In [7]:
for ens in range(nEns):
# for ens in ([0]):
    enss = ens+1
    modelDataFN = modelDataTemplate % ( code, enss, yr1, yr2 )
    print modelDataFN
    D = Model(None, modelDataFN)

    tas = np.array( D.loadData( "TREFHT" ) )
    nRec, nLat, nLon = tas.shape
    print nRec, nLat, nLon

    if ens < 1:
        print "Loading time, defining work vars"
        time, timeCF = D.loadData( "time" )
#         print time[0], time[-1]
        Tpts = np.empty( (nRec, 4), dtype = 'float' )
        TptsSave = np.empty( (nEns, nSites, nRec, 4), dtype = 'float' )
        Tnew = np.empty( (nEns, nRec, nSites), dtype = 'float' )


    for awsIX in (range(nSites)):
    # for awsIX in ((0,1)):
#     for awsIX in ([0]):
        """ Basic AWS info """
        awsName = A.getName( awsIX )
        awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

        """ Perform the interpolation on tas from model metadata """
        closest = C[awsIX]
        dx = "%.1f km" % closest[0][-1]
        print awsName, dx
        for r in range(nRec):
            Tpts[r, :] = [ tas[ r, d[0], d[1] ] for d in zip( closest['ixLat'], closest['ixLon'] )]
        Tnew[ens,:,awsIX] = M.interpolate( Tpts, closest, awsLatLon )
        TptsSave[ens,awsIX,:,:] = Tpts

1pt5degC/daily/b.e11.BRCP26C5CNBDRD.f09_g16.1pt5degC.001.cam.h1.TREFHT.20710101-20801231_sub.nc
3650 30 45
Loading time, defining work vars
Swiss Camp 40.2 km
Crawford Point1 41.5 km
NASA-U 22.5 km
GITS 37.7 km
Humboldt 22.3 km
Summit 51.5 km
Tunu-N 30.4 km
DYE-2 4.8 km
JAR1 28.6 km
Saddle 53.7 km
South Dome 52.4 km
NASA-E 8.7 km
Crawford Point2 40.1 km
NGRIP 20.4 km
NASA-SE 4.4 km
KAR 51.9 km
JAR2 17.2 km
KULU 34.3 km
JAR3 18.8 km
Aurora 28.9 km
Petermann GL 13.1 km
Petermann ELA 51.2 km
NEEM 28.8 km
1pt5degC/daily/b.e11.BRCP26C5CNBDRD.f09_g16.1pt5degC.002.cam.h1.TREFHT.20710101-20801231_sub.nc
3650 30 45
Swiss Camp 40.2 km
Crawford Point1 41.5 km
NASA-U 22.5 km
GITS 37.7 km
Humboldt 22.3 km
Summit 51.5 km
Tunu-N 30.4 km
DYE-2 4.8 km
JAR1 28.6 km
Saddle 53.7 km
South Dome 52.4 km
NASA-E 8.7 km
Crawford Point2 40.1 km
NGRIP 20.4 km
NASA-SE 4.4 km
KAR 51.9 km
JAR2 17.2 km
KULU 34.3 km
JAR3 18.8 km
Aurora 28.9 km
Petermann GL 13.1 km
Petermann ELA 51.2 km
NEEM 28.8 km
1pt5degC/daily/b.e1

Create a netCDF file of the interpolated data
====

In [11]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Interpolated temperature data"
ncfile.period = suff

# define dimensions
time_dim = ncfile.createDimension('time', None)
ens_dim = ncfile.createDimension('ensemble', nEns)
stn_dim = ncfile.createDimension('station', nSites)
pts_dim = ncfile.createDimension('pts', 4)

# define and write time
timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis

timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# define and write tas
# tasVar = ncfile.createVariable('tas', np.float32, ('time','station'))
tasVar = ncfile.createVariable('tas', np.float32, ('ensemble','time','station'))
tasVar.units = "K"
tasVar.description = "Surface temperature, interpolated"
tasVar.long_name = "Reference height temperature"
tasVar[:,:] = Tnew

# define and write saved Tpts
ptsVar = ncfile.createVariable('Tpts', np.float32, ('ensemble','station','time','pts'))
ptsVar.units = "K"
ptsVar.description = "Surface temperature, nearest grid points"
ptsVar.long_name = "Reference height temperature"
ptsVar[:] = TptsSave

ncfile.close()

In [9]:
newNCFN

'cesmlw_tas_closest_20710101-20801231.nc'